In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/kaigorodov/myprojects/MedConcNorm/')

In [3]:
import numpy as np
import pandas as pd
import pickle
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

from src.features.metrics import accuracy_top_K_pobs, sensitivity_area_K_range
from src.purpose_models.trainer import Trainer

from src.data.sentence_vectorizer import SentenceVectorizer

In [94]:
from sklearn.datasets import make_classification
X, Y = make_classification(n_samples=5000, n_features=15, n_informative=12,
                           n_classes=10, n_clusters_per_class=5, random_state=0)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.7)
X.shape, Y.shape

((5000, 15), (5000,))

In [133]:
from gensim.utils import tokenize

# gensim.utils.tokenize(text, lowercase=False, deacc=False, errors='strict', to_lower=False, lower=False)

sent = 'this is verybad, Sentece'


['this', 'is', 'verybad', 'sentece']

In [95]:
# smm4h21_train = pd.read_csv('../data/interim/smm4h21/train.csv')
# smm4h21_test = pd.read_csv('../data/interim/smm4h21/test.csv')

# import os
# labels = pd.read_csv('../data/interim/meddra_codes_terms_synonims.csv')
# labels = labels['CODE']
# meddra_labels = {v:k for k, v in enumerate(labels.unique())}

# results = {
#     'vectorizer': [],
#     'train_model': [],
#     'smm4h21': [],
#     'smm4h17': [],
#     'psytar': [],
#     'cadec': [],
# }

# SIZE = 100
# sv = SentenceVectorizer()
# for vectorizer_name in sv.get_availables_vectorizers():
#     print(f"vectorizer: {vectorizer_name}")
#     results['vectorizer'] += [vectorizer_name] * 4
#     path = '../data/interim/'
#     for name_train in os.listdir(path):

#         if name_train not in ['smm4h17', 'smm4h21', 'psytar', 'cadec']:
#             continue
#         print(f"work with {name_train}")
#         results['train_model'].append(name_train)

#         folder = os.path.join(path, name_train)
        
#         corpus_train = folder + '/train.csv'
#         train = pd.read_csv(corpus_train)
#         train = sv.vectorize(train, vectorizer_name=vectorizer_name) 
#         X_train, y_train = train['term_vec'], train['code']
#         X_train = pd.DataFrame([pd.Series(x) for x in X_train])
#         y_train = y_train.apply(lambda x: int(meddra_labels[x]))
        
#         corpus_test = folder + '/train.csv'
#         test = pd.read_csv(corpus_test)
#         test = sv.vectorize(test, vectorizer_name=vectorizer_name) 
#         X_test, y_test = test['term_vec'], test['code']
#         X_test = pd.DataFrame([pd.Series(x) for x in X_test])
#         y_test = y_test.apply(lambda x: int(meddra_labels[x]))
#         break
#     break

In [96]:
trainer = Trainer()
trainer.train_model(X_train, y_train)

In [101]:
trainer.accuracy(X_test[:10], y_test[:10], k=4)

0.5

In [98]:
trainer.model.score(X_test[:10], y_test[:10])

0.3

In [14]:
results_df = pd.DataFrame()
sensitivity_model = []
for max_iter in [100, 200, 500, 800, 1000]:

    clf = make_pipeline(StandardScaler(),
                        SGDClassifier(max_iter=max_iter, tol=1e-3, loss='log'))
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)
    sensitivity_model.append(sensitivity_area_K_range(y_test, y_pred))
                             
    print('unique classes in y:', np.unique(np.concatenate([y_train, y_test])).shape[0], end='\n'*2)

    print("default scoring:", clf.score(X_test, y_test))
    print("sensitivity:", sensitivity_model[-1])

    k_values, k_estims = [], []
    for k in range(1, 11):
        k_values.append(k)
        k_estims.append(accuracy_top_K_pobs(y_test, y_pred, k=k))
        #print(f"accuracy@{k}:", k_estims[-1])
    results_df[f"k"] = k_values
    results_df[f"accuracy_{max_iter}"] = k_estims
    
sensitivity_model_df = pd.DataFrame({'model': results_df.columns[1:], 'sense': sensitivity_model})
results_df = results_df.set_index('k')
results_df

/home/kaigorodov/.local/lib/python3.6/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


unique classes in y: 475

default scoring: 0.7768691588785047
sensitivity: 0.0


/home/kaigorodov/.local/lib/python3.6/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


unique classes in y: 475

default scoring: 0.7774532710280374
sensitivity: 0.0
unique classes in y: 475

default scoring: 0.7885514018691588
sensitivity: 0.0
unique classes in y: 475

default scoring: 0.7646028037383178
sensitivity: 0.0


KeyboardInterrupt: 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
sns.lineplot(data=results_df)

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(data=sensitivity_model_df, x='model', y='sense')